# Capstone Data Wrangling - FEMA Data

In [1]:
import pandas as pd
import requests
from urllib.request import urlretrieve
import numpy as np

## Import all Datasets

Once we've imported the relevant packages, it's time to retrieve our datasets. These will be coming from the FEMA website an their urls will be shown below. This type of retrieval allows for replicability--as opposed to downloading each .csv and then uploading them using a file path that only makes sense on my machine.

In [2]:
# First dataset : Summary of all disasters

url = 'https://www.fema.gov/api/open/v1/DisasterDeclarationsSummaries.csv'
file = 'DisasterDeclarationsSummaries.csv'
urlretrieve(url, file)

# Second dataset : Assistance data to home owners

url2 = 'https://www.fema.gov/api/open/v1/HousingAssistanceOwners.csv'
file2 = 'HousingAssistanceOwners.csv'
urlretrieve(url2, file2)

# Third dataset : Assistance data to renters

url3 = 'https://www.fema.gov/api/open/v1/HousingAssistanceRenters.csv'
file3 = 'HousingAssistanceRenters.csv'
urlretrieve(url3, file3)

### Wrangling the Disaster Declarations Summaries df

In [4]:
dates = ['declarationDate','incidentBeginDate','incidentEndDate','disasterCloseOutDate', 'lastRefresh']

dds_df = pd.read_csv(file, sep = ',', parse_dates = dates)

print(dds_df.columns)

dds_df = dds_df.drop(columns = ['declaredCountyArea','hash','placeCode','lastRefresh'])
#dds_df = dds_df.set_index('disasterNumber')

print("summaries shape : "+str(dds_df.shape))

dds_dup = dds_df.drop_duplicates(subset=['disasterNumber'])


dds_dup = dds_dup.set_index('disasterNumber')
dds_dup = dds_dup[['state','declarationDate','fyDeclared','disasterType','incidentType','title',
                   'ihProgramDeclared','iaProgramDeclared','paProgramDeclared', 'hmProgramDeclared',
                   'incidentBeginDate', 'incidentEndDate', 'disasterCloseOutDate']]
print("summaries shape after duplicates drop : "+str(dds_dup.shape))

Index(['disasterNumber', 'ihProgramDeclared', 'iaProgramDeclared',
       'paProgramDeclared', 'hmProgramDeclared', 'state', 'declarationDate',
       'fyDeclared', 'disasterType', 'incidentType', 'title',
       'incidentBeginDate', 'incidentEndDate', 'disasterCloseOutDate',
       'declaredCountyArea', 'placeCode', 'hash', 'lastRefresh'],
      dtype='object')
summaries shape : (49614, 14)
summaries shape after duplicates drop : (3865, 13)


In [5]:
dds_dup['disasterLength'] = dds_dup['incidentEndDate'] - dds_dup['incidentBeginDate']

This first dataset used quite a few different date variables, so to make things a bit cleaner, I just identified them and put them in a list 'dates' so that the pd.read_csv() function would be a bit easier to read.
Then we import the csv with the home owners' information:

### Wrangling the Housing Assistance Data Owners df

In [6]:
own_df = pd.read_csv(file2, sep = ',', parse_dates = ['lastRefresh'], low_memory=False)

own_df = own_df.drop(columns = ['hash','lastRefresh'])
own_df = own_df.dropna(subset=['zipCode'])
own_df = own_df[own_df['zipCode']!='00000']
own_df = own_df[own_df['zipCode']!=0]
print(own_df.columns)
print(own_df.shape)

sub_own = own_df.iloc[:,[0,3,4,5,7,9,14,15,16,17,18,19,20,21,22]]
#remaining columns own_remain = own_df.iloc[1,2,6,8,10,11,12,13]

sub_own.info()

own_df = own_df.set_index(['disasterNumber','zipCode','city'])

Index(['disasterNumber', 'state', 'county', 'city', 'zipCode',
       'validRegistrations', 'averageFemaInspectedDamage', 'totalInspected',
       'totalDamage', 'noFemaInspectedDamage',
       'femaInspectedDamageBetween1And10000',
       'femaInspectedDamageBetween10001And20000',
       'femaInspectedDamageBetween20001And30000',
       'femaInspectedDamageGreaterThan30000', 'approvedForFemaAssistance',
       'totalApprovedIhpAmount', 'repairReplaceAmount', 'rentalAmount',
       'otherNeedsAmount', 'approvedBetween1And10000',
       'approvedBetween10001And25000', 'approvedBetween25001AndMax',
       'totalMaxGrants'],
      dtype='object')
(89067, 23)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 89067 entries, 0 to 89140
Data columns (total 15 columns):
disasterNumber                  89067 non-null int64
city                            89062 non-null object
zipCode                         89067 non-null object
validRegistrations              89067 non-null int64
totalInspecte

### Wrangling the Housing Assistance Data Renters df

In [7]:
rent_df = pd.read_csv(file3, sep = ',', parse_dates = ['lastRefresh'], low_memory=False)

rent_df = rent_df.drop(columns = ['hash','lastRefresh','state','county'])
rent_df = rent_df.rename(columns={'totalInspectedWithNoDamage': 'noFemaInspectedDamage'})
rent_df.repairReplaceAmount = rent_df.repairReplaceAmount.astype('float64')
rent_df = rent_df[rent_df['zipCode']!=0]
rent_df = rent_df[rent_df['zipCode']!='00000']

print(rent_df.columns)
#rent_df.head(5)

Index(['disasterNumber', 'city', 'zipCode', 'validRegistrations',
       'totalInspected', 'noFemaInspectedDamage', 'totalWithModerateDamage',
       'totalWithMajorDamage', 'totalWithSubstantialDamage',
       'approvedForFemaAssistance', 'totalApprovedIhpAmount',
       'repairReplaceAmount', 'rentalAmount', 'otherNeedsAmount',
       'approvedBetween1And10000', 'approvedBetween10001And25000',
       'approvedBetween25001AndMax', 'totalMaxGrants'],
      dtype='object')


In [8]:
rent_df = rent_df.dropna(subset=['zipCode'])
rent_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64453 entries, 0 to 64524
Data columns (total 18 columns):
disasterNumber                  64453 non-null int64
city                            64447 non-null object
zipCode                         64453 non-null object
validRegistrations              64453 non-null int64
totalInspected                  64453 non-null int64
noFemaInspectedDamage           64453 non-null int64
totalWithModerateDamage         64453 non-null int64
totalWithMajorDamage            64453 non-null int64
totalWithSubstantialDamage      64453 non-null int64
approvedForFemaAssistance       64453 non-null int64
totalApprovedIhpAmount          64453 non-null float64
repairReplaceAmount             64453 non-null float64
rentalAmount                    64453 non-null float64
otherNeedsAmount                64453 non-null float64
approvedBetween1And10000        64453 non-null int64
approvedBetween10001And25000    64453 non-null int64
approvedBetween25001AndMax      6

In [9]:
sub_rent = rent_df.iloc[:,[0,1,2,3,4,5,9,10,11,12,13,14,15,16,17]]
# remaining columns rent_remain = rent_df.iloc[6,7,8]

sub_rent.info()

rent_df = rent_df.set_index(['disasterNumber','zipCode','city'])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64453 entries, 0 to 64524
Data columns (total 15 columns):
disasterNumber                  64453 non-null int64
city                            64447 non-null object
zipCode                         64453 non-null object
validRegistrations              64453 non-null int64
totalInspected                  64453 non-null int64
noFemaInspectedDamage           64453 non-null int64
approvedForFemaAssistance       64453 non-null int64
totalApprovedIhpAmount          64453 non-null float64
repairReplaceAmount             64453 non-null float64
rentalAmount                    64453 non-null float64
otherNeedsAmount                64453 non-null float64
approvedBetween1And10000        64453 non-null int64
approvedBetween10001And25000    64453 non-null int64
approvedBetween25001AndMax      64453 non-null int64
totalMaxGrants                  64453 non-null int64
dtypes: float64(4), int64(9), object(2)
memory usage: 7.9+ MB


## Merge Owner and Renter data

the objective here will be to 1) merge these two df by disasterNumber, then zipCode, and then city, and then 2) treat certain columns in the new df as series of sums and others as standalone columns. 

In [11]:
# first we deal with the summed columns
# sub_own and sub_rent, subsets of the bigger datasets, were created in earlier blocks for this purpose

print(str(sub_own.shape) +'\n' + str(sub_rent.shape))

total_sub = pd.concat([sub_own,sub_rent]).groupby(['disasterNumber','zipCode','city']).sum().reset_index()

print(total_sub.shape) # I made a habit of checking 'shape' in order to gauge whether different 
                       # merging strategies were taking me in the right direction

print(total_sub.info())

(89067, 15)
(64453, 15)
(92477, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92477 entries, 0 to 92476
Data columns (total 15 columns):
disasterNumber                  92477 non-null int64
zipCode                         92477 non-null object
city                            92477 non-null object
validRegistrations              92477 non-null int64
totalInspected                  92477 non-null int64
noFemaInspectedDamage           92477 non-null int64
approvedForFemaAssistance       92477 non-null int64
totalApprovedIhpAmount          92477 non-null float64
repairReplaceAmount             92477 non-null float64
rentalAmount                    92477 non-null float64
otherNeedsAmount                92477 non-null float64
approvedBetween1And10000        92477 non-null int64
approvedBetween10001And25000    92477 non-null int64
approvedBetween25001AndMax      92477 non-null int64
totalMaxGrants                  92477 non-null int64
dtypes: float64(4), int64(9), object(2)
memory usa

In [12]:
total_sub = total_sub.set_index(['disasterNumber','zipCode','city'])
print("shape of sum df for owner and renter payment data : " + str(total_sub.shape) + '\n')

# below are the columns that didn't need to be added together but still need to be included in the final df

own_remain = own_df.reset_index().iloc[:,[0,1,2,3,4,6,8,10,11,12,13]].set_index(['disasterNumber','zipCode','city'])
print("shape of remaining relevant owner payment df columns : " + str(own_remain.shape) + '\n')

rent_remain = rent_df.reset_index().iloc[:,[0,1,2,6,7,8]].set_index(['disasterNumber','zipCode','city'])
print("shape of remaining relevant renter payment df columns : " + str(rent_remain.shape) + '\n')

shape of sum df for owner and renter payment data : (92477, 12)

shape of remaining relevant owner payment df columns : (89067, 8)

shape of remaining relevant renter payment df columns : (64453, 3)



In [13]:
total_sub = total_sub[~total_sub.index.duplicated()]
own_remain = own_remain[~own_remain.index.duplicated()]
rent_remain = rent_remain[~rent_remain.index.duplicated()]

In [14]:
#list of the dfs I want to combine, all of which share common indices

df_list = [total_sub.reset_index(),own_remain.reset_index(),rent_remain.reset_index()]
merge_df = df_list[0]
for df_ in df_list[1:]:
    merge_df = merge_df.merge(df_, on=['disasterNumber','zipCode','city'], how = 'outer')

merge_df = merge_df.set_index(['disasterNumber','zipCode','city'])

#df = pd.merge(new_df1, rent_remain.reset_index(), on = (['disasterNumber','zipCode','city']))

print(merge_df.info())
merge_df.head(25)


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 92488 entries, (1539, 02769, REHOBOTH) to (4337, 33756, nan)
Data columns (total 23 columns):
validRegistrations                         92477 non-null float64
totalInspected                             92477 non-null float64
noFemaInspectedDamage                      92477 non-null float64
approvedForFemaAssistance                  92477 non-null float64
totalApprovedIhpAmount                     92477 non-null float64
repairReplaceAmount                        92477 non-null float64
rentalAmount                               92477 non-null float64
otherNeedsAmount                           92477 non-null float64
approvedBetween1And10000                   92477 non-null float64
approvedBetween10001And25000               92477 non-null float64
approvedBetween25001AndMax                 92477 non-null float64
totalMaxGrants                             92477 non-null float64
state                                      76276 non-null object

validRegistrations  totalInspected  \
disasterNumber zipCode city                                                     
1539           02769   REHOBOTH                           1.0             1.0   
               03982   PUNTA GORDA                        1.0             1.0   
               11111   INDIAN LAKE ESTATE                 1.0             1.0   
               12825   ORLANDO                            1.0             1.0   
               30950   PUNTA GORDA                        1.0             1.0   
               31818   ORLANDO                            1.0             1.0   
               32008   BRANFORD                           2.0             2.0   
               32033   ELKTON                             5.0             3.0   
               32080   SAINT AUGUSTINE                   42.0            31.0   
               32082   PONTE VEDRA                        1.0             1.0   
                       PONTE VEDRA BEACH                  4.0             4.0   
               32084   SAINT AUGUSTINE                   76.0            57.0   
               32085   SAINT AUGUSTINE                    4.0             4.0   
               32086   SAINT AUGUSTINE                   50.0            27.0   
               32091   STARKE                             1.0             1.0   
               32092   SAINT AUGUSTINE                   13.0            12.0   
               32095   SAINT AUGUSTINE                   13.0            11.0   
               32102   ASTOR                             21.0            21.0   
               32105   BARBERVILLE                        3.0             3.0   
               32110   BUNNEL                             1.0             1.0   
                       BUNNELL                          113.0            91.0   
                       ESPANOLA                           2.0             2.0   
               32114   DAYTONA                            1.0             1.0   
                       DAYTONA BEACH                    950.0          1460.0   
               32115   DAYTONA BEACH                      3.0             3.0   

                                           noFemaInspectedDamage  \
disasterNumber zipCode city                                        
1539           02769   REHOBOTH                              1.0   
               03982   PUNTA GORDA                           0.0   
               11111   INDIAN LAKE ESTATE                    1.0   
               12825   ORLANDO                               0.0   
               30950   PUNTA GORDA                           0.0   
               31818   ORLANDO                               0.0   
               32008   BRANFORD                              0.0   
               32033   ELKTON                                2.0   
               32080   SAINT AUGUSTINE                      10.0   
               32082   PONTE VEDRA                           0.0   
                       PONTE VEDRA BEACH                     1.0   
               32084   SAINT AUGUSTINE                      22.0   
               32085   SAINT AUGUSTINE                       3.0   
               32086   SAINT AUGUSTINE                      10.0   
               32091   STARKE                                1.0   
               32092   SAINT AUGUSTINE                       4.0   
               32095   SAINT AUGUSTINE                       3.0   
               32102   ASTOR                                 6.0   
               32105   BARBERVILLE                           1.0   
               32110   BUNNEL                                1.0   
                       BUNNELL                              33.0   
                       ESPANOLA                              1.0   
               32114   DAYTONA                               1.0   
                       DAYTONA BEACH                       833.0   
               32115   DAYTONA BEACH                         3.0   

                  

In [15]:
# reorder columns so that state and county are first and second, respectively, after the indices, and not mixed in with
# all the integer/float data

merge_df = merge_df[['state','county','validRegistrations','totalInspected','noFemaInspectedDamage','approvedForFemaAssistance',
         'totalApprovedIhpAmount','repairReplaceAmount','rentalAmount','otherNeedsAmount','approvedBetween1And10000',
         'approvedBetween10001And25000','approvedBetween25001AndMax','totalMaxGrants','averageFemaInspectedDamage',
         'totalDamage','femaInspectedDamageBetween1And10000','femaInspectedDamageBetween10001And20000',
         'femaInspectedDamageBetween20001And30000','femaInspectedDamageGreaterThan30000','totalWithModerateDamage',
         'totalWithMajorDamage','totalWithSubstantialDamage']]

In [16]:
print(merge_df.isna().sum())

state                                      16212
county                                     16212
validRegistrations                            11
totalInspected                                11
noFemaInspectedDamage                         11
approvedForFemaAssistance                     11
totalApprovedIhpAmount                        11
repairReplaceAmount                           11
rentalAmount                                  11
otherNeedsAmount                              11
approvedBetween1And10000                      11
approvedBetween10001And25000                  11
approvedBetween25001AndMax                    11
totalMaxGrants                                11
averageFemaInspectedDamage                 16212
totalDamage                                16212
femaInspectedDamageBetween1And10000        16212
femaInspectedDamageBetween10001And20000    16212
femaInspectedDamageBetween20001And30000    16212
femaInspectedDamageGreaterThan30000        16212
totalWithModerateDam

Now we attempt to merge these two DataFrames (the previously merged financial data and the summaries df) so that disaster summary information is available with each record of financial information. The Disaster Declaration Summaries df (dds_dup) acts more or less as a key that will have its information repeated every time the coinciding disasterNumber appears in a record.

In [21]:
print("disaster summaries shape : "+str(dds_dup.shape)+"\n merged df shape : "+str(merge_df.shape))

df = pd.merge(merge_df.reset_index(), dds_dup.drop(columns = 'state').reset_index(), on = 'disasterNumber')

print("\n final df shape : "+str(df.shape)+ " and, ")

for key, group_df in df.groupby('incidentType'):
    print("after merge, the group of type '{}' has {} rows".format(key,len(group_df)))
    
df = df.set_index(['disasterNumber','zipCode','city'])
df = df.sort_values('disasterNumber')
df.head(3)

disaster summaries shape : (3865, 14)
 merged df shape : (92488, 23)

 final df shape : (92488, 39) and, 
after merge, the group of type 'Earthquake' has 412 rows
after merge, the group of type 'Fire' has 1871 rows
after merge, the group of type 'Flood' has 7029 rows
after merge, the group of type 'Hurricane' has 51454 rows
after merge, the group of type 'Mud/Landslide' has 31 rows
after merge, the group of type 'Other' has 7 rows
after merge, the group of type 'Severe Storm(s)' has 29387 rows
after merge, the group of type 'Tornado' has 416 rows
after merge, the group of type 'Typhoon' has 1848 rows
after merge, the group of type 'Volcano' has 33 rows


state              county  \
disasterNumber zipCode city                                      
1539           02769   REHOBOTH         FL  Hampshire (County)   
               33922   CAPTIVA          FL        Lee (County)   
                       NORTH CAPTIVA   NaN                 NaN   

                                      validRegistrations  totalInspected  \
disasterNumber zipCode city                                                
1539           02769   REHOBOTH                      1.0             1.0   
               33922   CAPTIVA                       1.0             1.0   
                       NORTH CAPTIVA                 1.0             1.0   

                                      noFemaInspectedDamage  \
disasterNumber zipCode city                                   
1539           02769   REHOBOTH                         1.0   
               33922   CAPTIVA                          1.0   
                       NORTH CAPTIVA                    1.0   

                                      approvedForFemaAssistance  \
disasterNumber zipCode city                                       
1539           02769   REHOBOTH                             0.0   
               33922   CAPTIVA                              0.0   
                       NORTH CAPTIVA                        0.0   

                                      totalApprovedIhpAmount  \
disasterNumber zipCode city                                    
1539           02769   REHOBOTH                          0.0   
               33922   CAPTIVA                           0.0   
                       NORTH CAPTIVA                     0.0   

                                      repairReplaceAmount  rentalAmount  \
disasterNumber zipCode city                                               
1539           02769   REHOBOTH                       0.0           0.0   
               33922   CAPTIVA                        0.0           0.0   
                       NORTH CAPTIVA                  0.0           0.0   

                                      otherNeedsAmount       ...         \
disasterNumber zipCode city                                  ...          
1539           02769   REHOBOTH                    0.0       ...          
               33922   CAPTIVA                     0.0       ...          
                       NORTH CAPTIVA               0.0       ...          

                                      incidentType  \
disasterNumber zipCode city                          
1539           02769   REHOBOTH          Hurricane   
               33922   CAPTIVA           Hurricane   
                       NORTH CAPTIVA     Hurricane   

                                                                            title  \
disasterNumber zipCode city                                                         
1539           02769   REHOBOTH       TROPICAL STORM BONNIE AND HURRICANE CHARLEY   
               33922   CAPTIVA        TROPICAL STORM BONNIE AND HURRICANE CHARLEY   
                       NORTH CAPTIVA  TROPICAL STORM BONNIE AND HURRICANE CHARLEY   

                                      ihProgramDeclared  iaProgramDeclared  \
disasterNumber zipCode city                                                  
1539           02769   REHOBOTH                       0                  0   
               33922   CAPTIVA                        0                  0   
                       NORTH CAPTIVA                  0                  0   

                                      paProgramDeclared  hmProgramDeclared  \
disasterNumber zipCode city                                                  
1539           02769   REHOBOTH                       1                  1   
               33922   CAPTIVA                        1                  1   
                       NORTH CAPTIVA                  1                  1   

                                       incidentBeginDate     incidentEndDate  \
disasterNumber zipCode city              

## Handling nulls

In [22]:
df.state = df.state.fillna(method = 'ffill')
df.county = df.county.fillna(method = 'ffill')

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 92488 entries, (1539, 02769, REHOBOTH) to (4407, 96956, BUTTE VALLEY)
Data columns (total 36 columns):
state                                      92488 non-null object
county                                     92488 non-null object
validRegistrations                         92477 non-null float64
totalInspected                             92477 non-null float64
noFemaInspectedDamage                      92477 non-null float64
approvedForFemaAssistance                  92477 non-null float64
totalApprovedIhpAmount                     92477 non-null float64
repairReplaceAmount                        92477 non-null float64
rentalAmount                               92477 non-null float64
otherNeedsAmount                           92477 non-null float64
approvedBetween1And10000                   92477 non-null float64
approvedBetween10001And25000               92477 non-null float64
approvedBetween25001AndMax                 92477 non-null

### Handling null values

In [25]:
df.iloc[:,2:23] = df.iloc[:,2:23].fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 92488 entries, (1539, 02769, REHOBOTH) to (4407, 96956, BUTTE VALLEY)
Data columns (total 36 columns):
state                                      92488 non-null object
county                                     92488 non-null object
validRegistrations                         92488 non-null float64
totalInspected                             92488 non-null float64
noFemaInspectedDamage                      92488 non-null float64
approvedForFemaAssistance                  92488 non-null float64
totalApprovedIhpAmount                     92488 non-null float64
repairReplaceAmount                        92488 non-null float64
rentalAmount                               92488 non-null float64
otherNeedsAmount                           92488 non-null float64
approvedBetween1And10000                   92488 non-null float64
approvedBetween10001And25000               92488 non-null float64
approvedBetween25001AndMax                 92488 non-null

##### Rewrite this block too

something something something

The next task will be to identify and deal with outliers in our data. The .describe() will produce a nice matrix of quartile metrics. Only certain columns are needed for this assessment, so we'll slice those to examine more easily:

In [1]:
relevant_cols = ['validRegistrations',
       'averageFemaInspectedDamage', 'totalInspected', 'totalDamage',
       'noFemaInspectedDamage', 'femaInspectedDamageBetween1And10000',
       'femaInspectedDamageBetween10001And20000',
       'femaInspectedDamageBetween20001And30000',
       'femaInspectedDamageGreaterThan30000', 'approvedForFemaAssistance',
       'totalApprovedIhpAmount', 'repairReplaceAmount', 'rentalAmount',
       'otherNeedsAmount', 'approvedBetween1And10000',
       'approvedBetween10001And25000', 'approvedBetween25001AndMax',
       'totalMaxGrants','disasterLength']

df[relevant_cols].describe()

NameError: name 'df' is not defined

In [2]:
relevant_cols

['validRegistrations',
 'averageFemaInspectedDamage',
 'totalInspected',
 'totalDamage',
 'noFemaInspectedDamage',
 'femaInspectedDamageBetween1And10000',
 'femaInspectedDamageBetween10001And20000',
 'femaInspectedDamageBetween20001And30000',
 'femaInspectedDamageGreaterThan30000',
 'approvedForFemaAssistance',
 'totalApprovedIhpAmount',
 'repairReplaceAmount',
 'rentalAmount',
 'otherNeedsAmount',
 'approvedBetween1And10000',
 'approvedBetween10001And25000',
 'approvedBetween25001AndMax',
 'totalMaxGrants',
 'disasterLength']

## Handling Outliers

Originally, I used a for loop to produce a printout, a series of statements telling us for each predictor in question, how many records were 1+, 2+, or 3+ standard deviations above the mean for that predictor? One line, for example, looked like this example:

"771 rows where validRegistrations is 1 standard deviation(s) above the mean."

This printout gave us a decent view of the extremes of our data, but was tedious to read and clearly not as asset for obtaining the information we wanted in a timely manner. A tabular format of this information would look cleaner, so I set out to create that using lists:

In [27]:
list1 = []
list2 = []
list3 = []

for col in relevant_cols:
    outliers1 = df[df[col] > df[col].mean() + (1 * df[col].std())] #data points 1 or more std dev away from the mean
    list1.append(len(outliers1.index))
    
    outliers2 = df[df[col] > df[col].mean() + (2 * df[col].std())] #data points 2 or more std dev away from the mean
    list2.append(len(outliers2.index))
    
    outliers3 = df[df[col] > df[col].mean() + (3 * df[col].std())] #data points 3 or more std dev away from the mean
    list3.append(len(outliers3.index))

In [28]:
outlier_df = pd.DataFrame(
    {'1+ std dev': list1,
     '2+ std dev': list2,
     '3+ std dev': list3
    }, index = relevant_cols)

print(outlier_df)

                                         1+ std dev  2+ std dev  3+ std dev
validRegistrations                             2736        1233         833
averageFemaInspectedDamage                        1           1           1
totalInspected                                 2982        1760        1151
totalDamage                                    1081         525         356
noFemaInspectedDamage                          2609        1563        1091
femaInspectedDamageBetween1And10000            3734        2161        1380
femaInspectedDamageBetween10001And20000        1207         623         408
femaInspectedDamageBetween20001And30000         633         369         259
femaInspectedDamageGreaterThan30000             749         413         290
approvedForFemaAssistance                      2228        1256         805
totalApprovedIhpAmount                          874         459         311
repairReplaceAmount                            1513         795         518
rentalAmount

As expected, the table is much easier to read. That there are any instances of data ranging 3+ standard deviations from the mean at all is interesting, but with nearly 100,000 records, 100-200 incidents shouldn't be too big of a deal. It really just begins to tell us about the shape of our data--potentially very long-tailed.

In [ ]:
# Save this df so we can use it later in different notebooks.
df.to_csv('FEMA.csv',sep = '\t',index = ['disasterNumber','zipCode','city'])